# Prepare Data From CSV Recording

## Load File

Data is loaded from a CSV recording file, accepted through an input prompt. This includes all positional data related to the 6 trackers (HMD, Left Controller, Right Controller, Waist, Left Foot, Right Foot).

'Data is loaded into a Pandas dataframe. The primary tracking data is then extracted, leaving extraneous data such as booleans for button presses.

The extracted columns are then concatenated into a new dataframe, and the columns are renamed for ease of reading.

The columns are reorded in the order of head/r_controller/l_controller/waist/r_foot/l_foot.

The new trimmed file is written to a directory (/test_data or /train_data), for further manipulation and loading into the model.


In [1]:
import pandas as pd


#Read in CSV
def GetRecording(path):
    recording_path = "../recordings/"
    file_name = input("Input Recording File Name")
    try:
        dataframe = pd.read_csv(recording_path + file_name + ".csv")
        return dataframe, file_name
    except: 
        print("Error Reading File: Check Spelling and Try Again")
        return 0
    
    
#Seperate each tracker to seperate dataframe

def GetColByName(dataframe):
    HMD = dataframe.loc[:, ["HMD0_tx", "HMD0_ty", "HMD0_tz"]]
    
    controller_1 = dataframe.loc[:, ['controller3_tx', 'controller3_ty', 'controller3_tz']]

    controller_2 = dataframe.loc[:, ['controller4_tx', 'controller4_ty', 'controller4_tz']]

    tracker_1 = dataframe.loc[:, ['generic7_tx', 'generic7_ty', 'generic7_tz']]

    tracker_2 = dataframe.loc[:, ['generic8_tx', 'generic8_ty', 'generic8_tz']]

    tracker_3 = dataframe.loc[:, ['generic9_tx', 'generic9_ty', 'generic9_tz']]

    joined = pd.concat([HMD,controller_1, controller_2, tracker_1 ,tracker_2 ,tracker_3], axis=1)
    return joined

def AssignTracker(dataframe):
    display(dataframe.iloc[0:1,:])
    trackerNum = 7
    for x in range(3):
        trackerStr = str(trackerNum)
        tracker = input('assign generic' + trackerStr)
        dataframe.rename(columns={'generic' + trackerStr + '_tx': tracker + '_x', 'generic' + trackerStr + '_ty': tracker + "_y", 'generic' + trackerStr + '_tz': tracker + '_z'}, inplace=True)
        trackerNum += 1
        
    controllerNum = 3
    for x in range(2):
        controllerStr = str(controllerNum)
        controller = input('assign controller' + controllerStr)
        dataframe.rename(columns={'controller' + controllerStr + '_tx': controller + '_x', 'controller' + controllerStr + '_ty': controller + "_y", 'controller' + controllerStr + '_tz': controller + '_z'}, inplace=True)
        controllerNum += 1
    dataframe.rename(columns={'HMD0_tx': 'head_x', 'HMD0_ty': 'head_y', 'HMD0_tz': 'head_z'}, inplace=True)
    return dataframe

def GetDirectory():
    choice = input("train or test data:")
    if choice == "test":
        output_path = "../test_data/"
    else:
        output_path = "../train_data/"
    return output_path

def OrderFeatures(dataframe):
    head = dataframe.loc[:, ['head_x', 'head_y', 'head_z']]
    l_controller = dataframe.loc[:, ['l_controller_x', 'l_controller_y', 'l_controller_z']]
    r_controller = dataframe.loc[:, ['r_controller_x', 'r_controller_y', 'r_controller_z']]
    waist = dataframe.loc[:, ['waist_x', 'waist_y', 'waist_z']]
    r_foot = dataframe.loc[:, ['r_foot_x', 'r_foot_y', 'r_foot_z']]
    l_foot = dataframe.loc[:, ['l_foot_x', 'l_foot_y', 'l_foot_z']]
    reordered = pd.concat([head , r_controller, l_controller, waist, r_foot, l_foot], axis=1)
    return reordered

    
    
def WriteOutput(path, dataframe, filename):
    output_file = path + filename + "_trimmed.csv"
    dataframe.to_csv(output_file, index = False)
    print(file_name + " output to " + path)
    
    

# Run data trimming functions

In [41]:
recording_path = " ../recordings"

dataframe, file_name = GetRecording(recording_path)
joined = GetColByName(dataframe)
renamed = AssignTracker(joined)
path = GetDirectory()
reordered = OrderFeatures(renamed)
WriteOutput(path, reordered, file_name)


Input Recording File Name leg_raise_5


,HMD0_tx,HMD0_ty,HMD0_tz,controller3_tx,controller3_ty,controller3_tz,controller4_tx,controller4_ty,controller4_tz,generic7_tx,generic7_ty,generic7_tz,generic8_tx,generic8_ty,generic8_tz,generic9_tx,generic9_ty,generic9_tz
0,1.964998,158.131058,3.838474,-18.907917,92.860924,-9.813828,29.159588,80.373276,2.166218,3.899813,97.88607,6.474656,21.754217,11.036432,13.468838,-12.954676,11.488569,19.045788


assign generic7 waist
assign generic8 r_foot
assign generic9 l_foot
assign controller3 l_controller
assign controller4 r_controller
train or test data: test


leg_raise_5 output to ../test_data/


# Data Normalization

## Data Scaling

The new CSV is loaded into memory, chosen through an input prompt
The data is then split between the features (the HMD and controller tracking data), and the labels (the waist and foot trackers).
These are loaded into Numpy arrays to peform normaliztion. The output from OpenVR Recorder is upscaled by 100. To correct this the array is divided by 100

In [81]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
#from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
#import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


output_path = "../trim_output/"


#read in formatted CSV
def ReadCSV(path):
    file_name = input("Input File Name")
    file_list = []
    file_list.append(file_name)
    try:
        dataframe = pd.read_csv(path + file_name + ".csv")
        print("Dataframe created")
    except:
        print("Error Reading File")
    return dataframe, file_list

def SplitFeaturesLabels(dataframe):
    x = dataframe.iloc[:, 0:9]
    y = dataframe.iloc[:, 9:18]
    return x, y

#Load data into Numpy array
def LoadArray(x, y):
    x_array = np.array(x)
    y_array = np.array(y)
    return x_array, y_array


def NormalizeValues (x, y):
    x =  np.divide(x, 100)
    y =  np.divide(y, 100)
    return x, y

def SampleSize(x, y):
    x_samples = x[0:600,:]
    y_samples = y[0:600,:]
    return x_samples, y_samples

def RoundValues(x, y): 
    x_rounded = np.around(x, 3)
    y_rounded = np.around(y, 3)
    return x_rounded, y_rounded



    
    

    





# Create Training Data

In [82]:
train_path = "../train_data/"
#load train data from csv
train_dataframe, files = ReadCSV(train_path)

print(files)
#split features and labels into seperate dataframes
x_train_df, y_train_df = SplitFeaturesLabels(train_dataframe)

#convert features and labels to numpy array
x_train, y_train = LoadArray(x_train_df, y_train_df)

#Divide values in array by 100
x_samples, y_samples = NormalizeValues(x_train, y_train)

print(x_samples.shape, x_samples)
print(y_samples.shape, y_samples)

#x_train, x_test, y_train, y_test = train_test_split(x_train_normalized, y_train_normalized)





Input File Name walking_7_trimmed


Dataframe created
['walking_7_trimmed']
(600, 9) [[0.56456936 1.65256363 0.82371849 ... 0.35004543 0.81725983 0.93927994]
 [0.56244183 1.6511261  0.82760582 ... 0.35082798 0.81521797 0.94517693]
 [0.56021255 1.64923187 0.8309227  ... 0.3510099  0.81342445 0.95069672]
 ...
 [0.42225151 1.55701126 0.6327589  ... 0.21533564 0.75101753 0.71076973]
 [0.41390022 1.55681183 0.62624306 ... 0.20336998 0.75672852 0.69413803]
 [0.40570499 1.55652176 0.6195657  ... 0.19236187 0.76251534 0.67823952]]
(600, 9) [[0.56388847 0.9584938  0.82307373 ... 0.6701664  0.12568272 1.02205978]
 [0.5649939  0.95908966 0.8268927  ... 0.67017311 0.12555384 1.02190422]
 [0.5658884  0.95991432 0.83057884 ... 0.67164154 0.12405634 1.02306038]
 ...
 [0.4459425  0.94995354 0.64034004 ... 0.39146503 0.11636929 0.66756081]
 [0.43529736 0.9514679  0.63556534 ... 0.39199577 0.115468   0.66869583]
 [0.42703381 0.95304367 0.63249866 ... 0.39176373 0.11532265 0.66892319]]


# Scale Values

In [ ]:
from sklearn.preprocessing import MinMaxScaler

print(x_samples.max())
print(x_samples.min())

print(y_samples.max())
print(y_samples.min())

scaler =MinMaxScaler()
print(x_samples[0:1])
scaled = scaler.fit(x_samples)
print(scaler.transform(x_samples[0:1]))
print(scaler.inverse_transform(x_samples[0:1]))


# Reshape Training Data

In [3]:

def ReshapeData(x, y):
    x_reshaped = np.expand_dims(x, axis=1)
    y_reshaped = np.expand_dims(y, axis=1)

    return x_reshaped, y_reshaped


In [68]:
x, y = ReshapeData(x_samples, y_samples)

print(x.shape, y.shape)

print(x.shape[1])

print(x.shape[2])


(600, 1, 9) (600, 1, 9)
1
9


# Create Test / Validation Data

In [ ]:
#Create a single test data file

test_path = "../test_data/"


test_dataframe = ReadCSV(test_path)

#split features and labels into seperate dataframes
x_test_df, y_test_df = SplitFeaturesLabels(test_dataframe)

#convert features and labels to numpy array
x_test, y_test = LoadArray(x_test_df, y_test_df)

#Divide values in array by 100
x_test_normalized, y_test_normalized = NormalizeValues(x_test, y_test)

x_test_samples, y_test_samples = SampleSize(x_test_normalized, y_test_normalized)

x, y = ReshapeData(x_test_samples, y_test_samples)

print(x.shape, y_test.shape)

print(x_test.shape[1])

Input File Name walking_7_trimmed


Error Reading File


UnboundLocalError: local variable 'dataframe' referenced before assignment

# Combine Test and Train datasets

Combine all data sets in /train_data and /test_data into one, for more samples when training 

In [5]:
#import modules 

import os
import pandas as pd
import numpy as np


In [6]:

#Combine all datasets in a directory into one dataframe
def CombineDatasets(path):
    data_list = []
    for file in os.listdir(path):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"):
            df = pd.read_csv(path + filename)
            
            data_list.append(df)
         

    data_df = pd.concat(data_list, axis=0, ignore_index=True)
    return data_df



In [7]:

#combine data in a directory into two lists of x and y features
def DatasetsLists(path):
    x_list = []
    y_list = []
    file_list = []
    for file in os.listdir(path):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"):
            file_list.append(filename)
            df = pd.read_csv(path + filename)
            x_features, y_features = SplitFeaturesLabels(df)
            x_train, y_train = LoadArray(x_features, y_features)
            x_normalized, y_normalized = NormalizeValues(x_train, y_train)
            x_reshape, y_reshape = ReshapeData(x_normalized, y_normalized)
            x_list.append(x_reshape)
            y_list.append(y_reshape)
    return x_list, y_list, file_list



In [93]:
train_path = "../train_data/"
x, y, files = DatasetsLists(train_path)
print(files)

['arm_raise_1_trimmed.csv', 'arm_raise_3_trimmed.csv', 'boxing_1_trimmed.csv', 'crouch_walking_1_trimmed.csv', 'jumping_1_trimmed.csv', 'jumping_2_trimmed.csv', 'jumping_3_trimmed.csv', 'leg_raise_1_trimmed.csv', 'leg_raise_2_trimmed.csv', 'leg_raise_3_trimmed.csv', 'leg_raise_4_trimmed.csv', 'picking_up_1_trimmed.csv', 'running_on_spot_2_trimmed.csv', 'sitting_on_floor_1_trimmed.csv', 'sitting_standing_1_trimmed.csv', 'sitting_standing_2_trimmed.csv', 'sitting_standing_3_trimmed.csv', 'walking_1_train.csv', 'walking_2_test.csv', 'walking_3_trimmed.csv', 'walking_5_trimmed.csv', 'walking_6_trimmed.csv', 'walking_7_trimmed.csv']


Combine all data in the training data directory 

In [9]:
train_path = "../train_data/"

combined_train_dataframe = CombineDatasets(train_path)

x_train, y_train = SplitFeaturesLabels(combined_train_dataframe)
        
x_train_arr, y_train_arr = LoadArray(x_train, y_train)

#Divide values in array by 100
x_train_normalized, y_train_normalized = NormalizeValues(x_train_arr, y_train_arr)




x_train, y_train = ReshapeData(x_train_normalized, y_train_normalized)

print(x_train.shape, y_train.shape)


(16108, 1, 9) (16108, 1, 9)


Combine all data in the test data directory

In [10]:
test_path = "../test_data/"

combined_test_dataframe = CombineDatasets(test_path)

x_test, y_test = SplitFeaturesLabels(combined_test_dataframe)
        
x_test_arr, y_test_arr = LoadArray(x_test, y_test)

x_test_normalized, y_test_normalized = NormalizeValues(x_test_arr, y_test_arr)


x_test, y_test = ReshapeData(x_test_normalized, y_test_normalized)

print(x_test.shape)
print(x_test[0:1,:,:])
print(x_test_normalized.shape)


(5400, 1, 9)
[[[ 0.07488835  1.60079437 -0.06116605  0.29804235  0.82701195
   -0.13233584 -0.16509771  0.80914047 -0.10186529]]]
(5400, 9)


# Model Creation and Training

In [62]:
#import required modules

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, BatchNormalization
import tensorflow as tf
from keras.layers import LSTM, GRU
from tensorflow.keras.optimizers import SGD
from keras.layers.core import Dense, Activation, Dropout
from keras.losses import SparseCategoricalCrossentropy

tf.keras.backend.set_floatx('float64')


# Model Functions

In [94]:
#Fit model to two equally sized lists of x features and y labels 

def FitToList(x, y, model, epoch, b_size, verbose):
    for i in range(len(x)):
        print(i)
        print(x[i].shape)
        print(y[i].shape)
        model.fit(x[i], y[i], epochs=epoch,batch_size=b_size, verbose=verbose)
    return epoch, b_size
        
 # fit the model to a given set of features (x) and labels (y)    
def FitModel(x, y, model, epoch, b_size):
    model.fit(x, y, validation_split=0.33, epochs=epoch,batch_size = b_size)
    return epoch, b_size

    
def EvaluateModel(x, y, model):
    metrics = model.evaluate(x, y, batch_size=256)
    print("test loss, test acc:", metrics)
    return metrics
    
    
def PredictModel(x, y, model):
    predictions = model.predict(x)
    y_reshaped = y.reshape(-1,9)
    return predictions, y_reshaped


def DisplayPredictions(prediction, actual, range_1, range_2):
    if range_2 <= 0:
        r_1 = 0
        r_2 = len(prediction)
    else:
        r_1 = range_1
        r_2 = range_2
    print("predictions shape:", prediction.shape)
    prediction_df = pd.DataFrame(prediction, columns=["Waist_X", "Waist_Y", "Waist_Z", "R_Foot_X", "R_Foot_Y", "R_Foot_Z", "L_Foot_X", "L_Foot_Y", "L_Foot_Z"])
    actual_df = pd.DataFrame(actual, columns=["Waist_X", "Waist_Y", "Waist_Z", "R_Foot_X", "R_Foot_Y", "R_Foot_Z", "L_Foot_X", "L_Foot_Y", "L_Foot_Z"])
    print("Actual Values")
    display(actual_df[r_1:r_2])
    print("Predicited Values")
    display(prediction_df[r_1:r_2])
    return actual_df, prediction_df
    

        

# Model 1

In [8]:
sgd = SGD(learning_rate=0.001, momentum=0.8, decay=0.999, nesterov=False)

tf.keras.backend.set_floatx('float64')

model = Sequential()
model.add(GRU(16, return_sequences=True, input_shape=(5, 9)))
model.add(Dropout(0.2))
model.add(GRU(16, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(9, activation = "linear"))

model.compile(loss='mse', optimizer="adam")

print ('model compiled')

print (model.summary())

model compiled
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 1, 16)             1296      
                                                                 
 dropout (Dropout)           (None, 1, 16)             0         
                                                                 
 gru_1 (GRU)                 (None, 16)                1632      
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 9)                 153       
                                                                 
Total params: 3,081
Trainable params: 3,081
Non-trainable params: 0
_________________________________________________________________
None


# Model 2 

In [136]:
model_2 = Sequential()

opt = SGD(learning_rate = 0.01)

compile_params = ["mse", "sgd", "accuracy"]
#add one GRU layer of 64 cellss with input shape 1,9
model_2.add(GRU(64, input_shape=(1, 9)))
#model_2.add(Dropout(0.1))

#Add Batch Normalization layer
#model_2.add(BatchNormalization())


#Add dropout layer
#model_2.add(Dropout(0.2))

#Add Dense layer with 9 outputs
model_2.add(Dense(9))

print(model_2.summary())


#Compile model
model_2.compile(
    loss = compile_params[0],
    optimizer = opt,
    metrics = [compile_params[2]],
)



Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_14 (GRU)                (None, 64)                14400     
                                                                 
 dense_14 (Dense)            (None, 9)                 585       
                                                                 
Total params: 14,985
Trainable params: 14,985
Non-trainable params: 0
_________________________________________________________________
None


In [115]:
epochs, batch_size = FitModel(x, y, model_2, 1, 256)

ValueError: Data cardinality is ambiguous:
  x sizes: 606, 600, 600, 600, 606, 600, 600, 606, 600, 600, 600, 600, 600, 600, 606, 600, 600, 606, 606, 600, 600, 600, 600
  y sizes: 606, 600, 600, 600, 606, 600, 600, 606, 600, 600, 600, 600, 600, 600, 606, 600, 600, 606, 606, 600, 600, 600, 600
Make sure all arrays contain the same number of samples.

In [133]:
#Fit list of datasets to model

#training_x, training_y, model to train on, epoch count, batch size, ?verbose
epochs, batch_size = FitToList(x, y, model_2, 20, 1, 1)

0
(905, 1, 9)
(905, 1, 9)
Epoch 1/20
905/905 [==============================] - 1s 797us/step - loss: 0.0088 - accuracy: 0.9812
Epoch 2/20
905/905 [==============================] - 1s 804us/step - loss: 0.0013 - accuracy: 1.0000
Epoch 3/20
905/905 [==============================] - 1s 786us/step - loss: 8.8171e-04 - accuracy: 1.0000
Epoch 4/20
905/905 [==============================] - 1s 785us/step - loss: 6.2388e-04 - accuracy: 1.0000
Epoch 5/20
905/905 [==============================] - 1s 795us/step - loss: 4.4913e-04 - accuracy: 1.0000
Epoch 6/20
905/905 [==============================] - 1s 812us/step - loss: 3.3177e-04 - accuracy: 1.0000
Epoch 7/20
905/905 [==============================] - 1s 792us/step - loss: 2.5232e-04 - accuracy: 1.0000
Epoch 8/20
905/905 [==============================] - 1s 805us/step - loss: 1.9742e-04 - accuracy: 1.0000
Epoch 9/20
905/905 [==============================] - 1s 794us/step - loss: 1.5927e-04 - accuracy: 1.0000
Epoch 10/20
905/905 [=======

In [134]:

results = EvaluateModel(x_test, y_test, model_2)


22/22 [==============================] - 0s 3ms/step - loss: 0.0863 - accuracy: 0.7708
test loss, test acc: [0.08630823824485695, 0.7708260702830847]


In [135]:
predicted_values, actual_values = PredictModel(x_test, y_test, model_2)

toWriteActual, toWritePred = DisplayPredictions(predicted_values, actual_values, 0, 0)



predictions shape: (5400, 9)
Actual Values


,Waist_X,Waist_Y,Waist_Z,R_Foot_X,R_Foot_Y,R_Foot_Z,L_Foot_X,L_Foot_Y,L_Foot_Z
0,0.060439,1.022620,-0.098646,0.296409,0.111177,-0.002579,-0.105350,0.107512,0.008089
1,0.060260,1.022620,-0.098646,0.296409,0.111177,-0.002579,-0.105350,0.107512,0.008089
2,0.059955,1.022620,-0.098769,0.296418,0.111177,-0.002579,-0.105350,0.107433,0.008048
3,0.059620,1.022658,-0.099001,0.296458,0.111177,-0.002579,-0.105350,0.107417,0.007960
4,0.059283,1.022695,-0.099210,0.296351,0.111177,-0.002579,-0.105444,0.107411,0.007911
...,...,...,...,...,...,...,...,...,...
5395,-0.266650,1.028392,-0.104049,-0.055547,0.211484,0.045802,-0.325084,0.122200,0.065459
5396,-0.263761,1.024829,-0.101151,-0.055403,0.215707,0.044948,-0.325417,0.121903,0.065412
5397,-0.271885,1.032776,-0.107868,-0.054945,0.219580,0.044346,-0.325417,0.121905,0.065404
5398,-0.268666,1.028839,-0.104847,-0.054282,0.222948,0.044158,-0.325470,0.121905,0.065396


Predicited Values


,Waist_X,Waist_Y,Waist_Z,R_Foot_X,R_Foot_Y,R_Foot_Z,L_Foot_X,L_Foot_Y,L_Foot_Z
0,0.045796,0.995048,-0.046811,0.156399,0.119529,0.007963,-0.008249,0.139777,0.049285
1,0.045523,0.994901,-0.046653,0.156055,0.119601,0.007891,-0.008459,0.139713,0.049262
2,0.045237,0.994714,-0.046491,0.155726,0.119664,0.007807,-0.008677,0.139650,0.049253
3,0.044929,0.994533,-0.046392,0.155415,0.119700,0.007692,-0.008936,0.139602,0.049196
4,0.044505,0.994411,-0.046321,0.154952,0.119729,0.007614,-0.009329,0.139565,0.049123
...,...,...,...,...,...,...,...,...,...
5395,-0.216685,1.014766,-0.119630,-0.180516,0.136369,-0.074052,-0.271115,0.132533,-0.061018
5396,-0.216420,1.016081,-0.120537,-0.182189,0.137141,-0.075245,-0.270983,0.132110,-0.063776
5397,-0.216083,1.017301,-0.121268,-0.183811,0.137923,-0.076278,-0.270753,0.131686,-0.066311
5398,-0.215661,1.018524,-0.121838,-0.185240,0.138686,-0.077102,-0.270405,0.131278,-0.068514


# Write Results to File 

In [129]:
from datetime import datetime

def WriteResultToFile(path, pred, actual, model, metrics, file_list, epochs, b_size):
    try:
        #get current date for folder naming
        now = datetime.now()
        dt_string = now.strftime("%d_%m_%Y_%H_%M")
    
        #create new folder
        new_folder = "run_" + dt_string
        os.makedirs(path + new_folder)

        #loop over columns to split between actual and predicted
        for column in actual.columns:
            new_column = "A_" + column
            actual = actual.rename(columns={column : new_column})
        for column in pred.columns:
            new_column = "P_" + column
            pred = pred.rename(columns={column : new_column})

        #combine both actual and predicted dataframes
        results = pd.concat([actual, pred], axis=1)
    
        #create output path for csv write
        output_folder = results_path + new_folder
        results.to_csv(output_folder + "/prediction.csv", index = False, float_format='%.6f')
        metric_labels = ["Loss: ", "Accuracy: "]    
        compile_params_list = ["Loss Function: ", "Optimizer: ", "Metrics: "]
        #create new txt file to output model and training data summary
        with open(output_folder + '/summary.txt','w') as fh:
            model.summary(print_fn=lambda x: fh.write(x + '\n'))
            fh.write("Epochs: ")
            fh.write(str(epochs) + "\n")
            fh.write("Batch Size: ")
            fh.write(str(b_size) + "\n")
            fh.write("Metrics:\n")
            for i in range (len(metrics)):
                fh.write(metric_labels[i])
                fh.write(str(metrics[i]) + "\n")
            fh.write("Trained On:\n")
            for file in file_list:
                fh.write(file + "\n")
            for i in range (len(compile_params)):
                fh.write(compile_params_list[i])
                fh.write(compile_params[i] + "\n")
        print("File Output to " + new_folder)
    except: 
        
        print("Error Writing Results To File!")
        
        
    
    


In [137]:
results_path = "../results/"

WriteResultToFile(results_path, toWritePred, toWriteActual, model_2, results, files, epochs, batch_size) 



File Output to run_01_08_2022_00_35
